In [12]:
import pandas as pd

df = pd.read_csv('sinais_vitais_com_label_treino.txt', sep=',', header=0)
df

,id,pSist,pDiast,qPA,pulso,frequencia,gravidade,classe
0,1,18.907009,12.220855,8.733333,134.454047,17.972046,34.192304,2
1,2,14.698109,13.586879,8.614814,86.302237,4.168373,40.000000,2
2,3,16.428095,1.904802,-4.333333,138.880782,2.721256,27.122605,2
3,4,14.616501,13.700638,8.602531,51.356913,18.061923,47.188993,2
4,5,10.955766,9.485389,-0.000000,1.951730,14.033748,68.333333,3
...,...,...,...,...,...,...,...,...
795,796,11.478356,0.769978,-4.427394,102.221274,0.761861,43.185667,2
796,797,8.353375,1.093279,-5.561934,73.366682,9.924723,43.067405,2
797,798,3.232797,1.327912,0.000000,147.895901,0.303480,40.000000,2
798,799,13.457415,11.975263,7.818706,104.948033,10.421636,37.262834,2


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the training data
df = pd.read_csv('sinais_vitais_com_label_treino.txt', sep=',', header=0)
# X is columns 3, 4, 5 and 6
X = df.iloc[:, 2:6]
# y is column 7
y = df.iloc[:, 6]

# Create three different neural network models
model1 = MLPRegressor(hidden_layer_sizes=(50,), activation='relu', solver='adam', learning_rate_init=0.01, max_iter=200)
model2 = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', solver='sgd', learning_rate_init=0.01, max_iter=200)
model3 = MLPRegressor(hidden_layer_sizes=(50, 50), activation='tanh', solver='adam', learning_rate_init=0.01, max_iter=200)

# Train each model and calculate their RMSE
models = [model1, model2, model3]
for i, model in enumerate(models, start=1):
    model.fit(X, y)
    predictions = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    print(f'Model {i} RMSE: {rmse}')

# Assume model1 is the best model based on the lowest RMSE
best_model = model1

# Load the blind test data
blind_test_data = pd.read_csv('sinais_vitais_validacao.txt', sep=',', header=0)
X_blind_test = blind_test_data.iloc[:, 2:6]

# Use the best model to predict the severity values for the blind test data
blind_test_predictions = best_model.predict(X_blind_test)

# Save the predicted severity values to a CSV file
pd.DataFrame(blind_test_predictions).to_csv('predictions.csv', index=False, header=False)

Model 1 RMSE: 9.780516466523446
Model 2 RMSE: 15.557387912594525
Model 3 RMSE: 6.365698037039905


C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
# Load the actual values
actual_values = df['gravidade'][:len(blind_test_predictions)]

# Calculate the differences
differences = actual_values - blind_test_predictions

# Calculate the mean and standard deviation of the differences
mean_difference = differences.mean()
std_difference = differences.std()

print(f'Mean difference: {mean_difference}')
print(f'Standard deviation of differences: {std_difference}')

Mean difference: 1.6050085464421942
Standard deviation of differences: 9.499476487662104


In [24]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

hidden_layer_sizes_options = [(50,), (100,), (50, 50)]
activation_functions = ['identity', 'logistic', 'tanh', 'relu']
solvers = ['lbfgs', 'sgd', 'adam']
learning_rates = ['constant', 'invscaling', 'adaptive']
max_iterations = [100, 200, 300]

# learning_rate_inits is a list going from 0.001 to 0.05 in steps of 0.001
# learning_rate_inits = np.arange(0.001, 0.051, 0.001)
# fixed learning_rate_init of 0.01
learning_rate_inits = [0.01]

# Define the configurations for the MLPRegressor. Learning rate change only when solver is SGD
configurations = []
for hidden_layer_sizes in hidden_layer_sizes_options:
    for activation in activation_functions:
        for solver in solvers:
            for learning_rate in learning_rates:
                for max_iter in max_iterations:
                    for learning_rate_init in learning_rate_inits:
                        if solver == 'sgd':
                            configurations.append({'hidden_layer_sizes': hidden_layer_sizes, 'activation': activation, 'solver': solver, 'learning_rate': learning_rate, 'max_iter': max_iter, 'learning_rate_init': learning_rate_init})
                        else:
                            configurations.append({'hidden_layer_sizes': hidden_layer_sizes, 'activation': activation, 'solver': solver, 'max_iter': max_iter, 'learning_rate_init': learning_rate_init})

best_rmse = np.inf
best_model = None
best_config = None

# Iterate over the configurations
for config in configurations:
    try:
        # Create and train the MLPRegressor
        model = MLPRegressor(**config)
        model.fit(X, y)
    except ValueError:
        # If the configuration is invalid, skip it
        continue
    
    # Calculate the RMSE
    predictions = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    
    # Print the RMSE for this model
    # print(f'Model with {config} RMSE: {rmse}')
    
    # If this model has a lower RMSE than the current best, update the best RMSE and best model
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_config = config

print(f'Best model configuration: {best_config}')
print(f'Best model RMSE: {best_rmse}')

C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmat

Best model configuration: {'hidden_layer_sizes': (50, 50), 'activation': 'tanh', 'solver': 'adam', 'max_iter': 300, 'learning_rate_init': 0.01}
Best model RMSE: 5.250459384138478


In [23]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the hyperparameters
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [100, 200, 300],
    # 'learning_rate_init': np.arange(0.001, 0.051, 0.01)
}

# Create a MLPRegressor
mlp = MLPRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the GridSearchCV object
grid_search.fit(X, y)

# Get the best model
best_model = grid_search.best_estimator_

# Calculate the RMSE of the best model
predictions = best_model.predict(X)
rmse = np.sqrt(mean_squared_error(y, predictions))

print(f'Best model configuration: {grid_search.best_params_}')
print(f'Best model RMSE: {rmse}')

C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
168 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
168 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\henri\PycharmProjects\VictimSim\ven

Best model configuration: {'activation': 'tanh', 'hidden_layer_sizes': (50, 50), 'learning_rate': 'constant', 'max_iter': 200, 'solver': 'lbfgs'}
Best model RMSE: 9.79835844637706


C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
